In [4]:
import os
import re
import email
import pandas as pd
import quopri
import email
import re
import pandas as pd
import quopri
import os

In [ ]:
def extract_invoice_data(eml_file_path):
    with open(eml_file_path, 'r', encoding='utf-8') as f:
        msg = email.message_from_file(f)

        payload = msg.get_payload()
        if isinstance(payload, list):
            payload = payload[0].get_payload()

        body = quopri.decodestring(payload).decode('utf-8', errors='ignore')

        # Extract order number, time, and restaurant name
        order_number = re.search(r'Order number:\s*(\S+)', body)
        order_time = re.search(r'Order time:\s*([\d-]+\s[\d:]+)', body)
        restaurant_name = re.search(r"Your order from\s*(.+?)\swill", body)

        # Extract items and prices
        items = re.findall(r"(\d+)\sX\s([\w\s\-&]+)\s+Rs\.\s([\d,]+)", body)
        breakdown = re.findall(r"(Subtotal|Incl\. delivery fee|Platform fee|Rider tip|Discount|Order Total|Including [\d.]+% tax)\s+Rs\.\s([\d\.\-,]+)", body)

        # Format the extracted data
        return {
            'Order Number': order_number.group(1) if order_number else None,
            'Order Time': order_time.group(1) if order_time else None,
            'Restaurant Name': restaurant_name.group(1) if restaurant_name else None,
            'Items': items,  # List of tuples (Qty, Item Name, Price)
            'Breakdown': breakdown  # List of tuples (Breakdown Label, Price)
        }

directory = '/content/eml'  # Use the directory where your .eml file resides

data_list = []
for filename in os.listdir(directory):
    if filename.endswith(".eml"):
        eml_file_path = os.path.join(directory, filename)
        data = extract_invoice_data(eml_file_path)
        if data:
            # Separate items and breakdown for tabular display
            for qty, item, price in data['Items']:
                row = {
                    'Order Number': data['Order Number'],
                    'Order Time': data['Order Time'],
                    'Restaurant Name': data['Restaurant Name'],
                    'Qty': qty,
                    'Item': item.strip(),
                    'Price': price.replace(",", "")
                }
                data_list.append(row)
            for label, value in data['Breakdown']:
                data_list.append({
                    'Order Number': data['Order Number'],
                    'Order Time': data['Order Time'],
                    'Restaurant Name': data['Restaurant Name'],
                    'Qty': '',
                    'Item': label.strip(),
                    'Price': value.replace(",", "")
                })

df = pd.DataFrame(data_list)
print(df)


In [7]:
df.to_csv('FoodPanda_2.csv', index=False)